<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/day4_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API: Configuración inicial

Esta sección requiere una clave API con facturación habilitada. Comienza instalando e importando el SDK de la API de Gemini en Python.


In [1]:
%pip install -q -U 'google-generativeai>=0.8.3'

No necesitas reiniciar el kernel.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display


Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada como un secreto en Kaggle llamado GOOGLE_API_KEY.

Si aún no tienes una clave API, puedes obtener una en AI Studio. Consulta las instrucciones detalladas en la documentación.

Para que la clave esté disponible a través de los secretos de Kaggle, selecciona "Secrets" en el menú "Add-ons" y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.


In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


In [3]:
genai.configure(api_key='')

Explora los modelos disponibles

El "search grounding" (búsqueda basada en datos externos) es una herramienta disponible en los modelos de la serie -002. Encuentra un modelo que la soporte utilizando el endpoint `models.list`. También puedes encontrar más información sobre las capacidades de los modelos en la página de modelos.


In [4]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)


models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


Usa el "search grounding"

Para habilitar el "search grounding", especifica la herramienta `google_search_retrieval`. Como otras herramientas, puedes suministrarla como un parámetro al modelo (para usar en todos los turnos de chat o llamadas a `generate_content`), o puedes proporcionarla por turno utilizando `chat.send_message`.


In [5]:
# Solicita información sin "search grounding".
model = genai.GenerativeModel("gemini-1.5-flash-002")
response = model.generate_content("¿Cuándo y dónde será el próximo concierto de Taylor Swift?")
Markdown(response.text)


No puedo proporcionar la información exacta sobre el próximo concierto de Taylor Swift.  Las fechas y lugares de sus conciertos cambian constantemente a medida que se anuncian nuevas fechas de gira.

Para encontrar esta información, te recomiendo:

* **Visitar el sitio web oficial de Taylor Swift:** Este es el lugar más confiable para obtener información actualizada sobre sus próximos conciertos.
* **Revisar sitios de venta de entradas como Ticketmaster o Live Nation:**  Estos sitios suelen listar conciertos próximos con información de fechas y lugares.
* **Buscar en redes sociales oficiales de Taylor Swift:**  Ella suele anunciar nuevas fechas de conciertos en sus cuentas oficiales de Twitter, Instagram, etc.

Recuerda que la información que encuentres en otros sitios web o redes sociales no oficiales debe ser tratada con precaución, ya que podría ser inexacta o incluso fraudulenta.


Ahora intenta con el "search grounding" habilitado.

In [6]:
# Vuelve a ejecutar la misma consulta con el "search grounding" habilitado.
model = genai.GenerativeModel(
    "gemini-1.5-flash-002",
    tools="google_search_retrieval")
response = model.generate_content("¿Cuándo y dónde será el próximo concierto de Taylor Swift?")
rc = response.candidates[0]
Markdown(rc.content.parts[0].text)


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-002:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

Si recibes un error 429 (recurso agotado), es probable que estés usando una clave API gratuita. Puedes habilitar la facturación (lo que generará cargos) o probar las consultas en Google AI Studio siguiendo los enlaces "Open in AI Studio".

Metadatos de respuesta

Cuando se usa "search grounding", el modelo devuelve metadatos adicionales que incluyen enlaces a sugerencias de búsqueda, documentos de soporte e información sobre cómo se usaron esos documentos en la generación de la respuesta.

Cada "fragmento de grounding" representa información recuperada de Google Search utilizada en la solicitud de generación basada en datos externos. Siguiendo la URI, puedes acceder a la fuente.


In [ ]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)


Como parte de la respuesta, hay un bloque de contenido HTML estilizado independiente que puedes usar para enlazar sugerencias de búsqueda relacionadas con la generación.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

Los "supports" en los metadatos proporcionan una forma de correlacionar los fragmentos de grounding con el texto generado de salida.

In [ ]:
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    print(support)


Estos soportes pueden usarse para resaltar texto en la respuesta o construir tablas de notas al pie.

In [ ]:
import io
markdown_buffer = io.StringIO()
# Imprime el texto con marcadores de notas al pie.
markdown_buffer.write("Texto con soporte:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )
    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")
    markdown_buffer.write("\n\n")
# E imprime las notas al pie.
markdown_buffer.write("Citas:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")
Markdown(markdown_buffer.getvalue())


"Search grounding" dinámico

En un contexto donde no sabes de antemano si habilitar el "search grounding" o no, puedes proporcionar al modelo un umbral sobre el cual usará esta funcionalidad. Esto es útil en contextos conversacionales, donde no todos los turnos de conversación requieren datos de búsqueda para respaldar una respuesta.

Si sabes si habilitar la búsqueda para un turno específico, puedes proporcionar la herramienta explícitamente.


In [ ]:
nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
chat = nosearch_model.start_chat()
# Sin "search grounding".
r = chat.send_message("¡Hola, chatbot amigable!")
# Habilita la búsqueda solo para este turno.
r = chat.send_message(
    "¿Quién ganó la Copa Mundial de Cricket 2023?", tools="google_search_retrieval"
)
Markdown(r.text)


In [ ]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

O puedes dejar que la API de Gemini calcule la probabilidad de que la respuesta necesite soporte de búsqueda y definir el umbral a usar.

In [ ]:
# La puntuación de recuperación dinámica es una probabilidad, por lo que el umbral
# también debe estar dentro del rango [0, 1].
search_config = {
    "dynamic_retrieval_config": {"mode": "MODE_DYNAMIC", "dynamic_threshold": 0.5}
}
maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", tools={"google_search_retrieval": search_config}
)
chat = maybe_search_model.start_chat()
r = chat.send_message("¡Hola, chatbot amigable!")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Primer turno: {score=}")
r = chat.send_message("¿Quién ganó la Copa Mundial de Cricket 2023?")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Segundo turno: {score=}")
print()
display(Markdown(r.text))


In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

Lecturas adicionales

Cuando uses "search grounding", hay algunos requisitos específicos que debes seguir, como cuándo y cómo mostrar sugerencias de búsqueda y cómo usar los enlaces de grounding. Asegúrate de leer y seguir los detalles en la guía de capacidades de "search grounding" y la guía de sugerencias de búsqueda.
